In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import math
from numpy.linalg import inv

In [2]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    return (average, math.sqrt(variance))

In [3]:
al = 0.91
sig = 1
beta = 0.5
time = 500
Xsys = []
Ysys = []
x0 = np.random.normal(0,(sig**2/(1-al**2))**0.5) # first state
y0 = np.random.normal(0,((beta**2)*math.exp(x0))**0.5) #first observation
Xsys.append(x0)
Ysys.append(y0) 

for t in range(0,time-1):
    x = np.random.normal(al*Xsys[t],sig)
    y = np.random.normal(0,((beta**2)*math.exp(x))**0.5)
    Xsys.append(x)
    Ysys.append(y)
    

In [5]:
plt.figure(figsize=(15,15))
plt.title("Simulated Volatility Sequence", fontsize = 20)
plt.xlabel('time',fontsize=20)
#plt.ylabel('x-coordinate', fontsize=20)
plt.plot(np.arange(start=1, stop=time+1, step=1),Xsys,linestyle='-',color='blue',label='Volatility')
plt.plot(np.arange(start=1, stop=time+1, step=1),Ysys,'*', color='red',label='Observations')
plt.legend(loc='best',fontsize=15)
plt.show()

In [6]:
#SIS
T = 100 #time of filter
N = 1000 #number of particles
Xsis = np.zeros((T,N)) #matrix of particles for 100 times
Wsis = np.zeros((T,N)) #matrix of weight of particles for 100 times
Meansis = np.zeros(T) 
SDsis = np.zeros(T)

xsis0 = np.random.normal(0,(sig**2/(1-al**2))**0.5,N) #q(x1|y1)= u(x1) sample particle for first time
alsis0 = norm.pdf(Ysys[0], 0, ((beta**2)*np.exp(xsis0))**0.5)
wsis0 = alsis0/np.sum(alsis0)  #w1=g(y1|x1)
Xsis[0] = xsis0
Wsis[0] = wsis0

for t in range(0,T-1): 
    xsis = np.random.normal(al*Xsis[t],sig,N)  #q(xn|yn,xn-1) = f(xn|xn-1)
    alsis = norm.pdf(Ysys[t+1], 0, ((beta**2)*np.exp(xsis))**0.5) #calculate incremental weight \alpha_n = g(yn|xn)
    wsis = alsis*Wsis[t]/np.sum(alsis*Wsis[t]) #weight of each particl at time n #wn=\alpha_n * wn-1
    Xsis[t+1] = xsis
    Wsis[t+1] = wsis
    
for k in range(0,T):
    weighted_stats = weighted_avg_and_std(Xsis[k], Wsis[k])
    Meansis[k] = weighted_stats[0] #filtering mean time n
    SDsis[k] = weighted_stats[1] #filtering sd time n
    
plt.figure(figsize=(15,15))
plt.title("SIS Filtering Estimates", fontsize = 20)
plt.xlabel('time',fontsize=20)
#plt.ylabel('x-coordinate', fontsize=20)
plt.plot(np.arange(start=1, stop=T+1, step=1),Xsys[0:T],marker='+',linestyle='-',color='blue',label='True Volatility')
plt.plot(np.arange(start=1, stop=T+1, step=1),Meansis,linestyle='-',linewidth=1,color='red',label='Filter Mean')
plt.plot(np.arange(start=1, stop=T+1, step=1),Meansis+SDsis,linestyle='--',linewidth=0.7,color='red',label='+/- 1 S.D')
plt.plot(np.arange(start=1, stop=T+1, step=1),Meansis-SDsis,linestyle='--',linewidth=0.7,color='red')
plt.legend(loc='best',fontsize=15)
plt.show()

#SIS histogram of particle weights
fig, ax = plt.subplots(1,4, figsize=(15,8))
fig.suptitle('SIS')

ax[0].hist(Wsis[1], np.histogram(Wsis[1])[1])
ax[0].set_title("n=2", fontsize = 12)
ax[0].set_xlabel('Normalised Weights', fontsize=12)
ax[0].set_ylabel('Particle Count', fontsize=12)

ax[1].hist(Wsis[9], np.histogram(Wsis[9])[1])
ax[1].set_title("n=10", fontsize = 12)
ax[1].set_xlabel('Normalised Weights', fontsize=12)

ax[2].hist(Wsis[49], np.histogram(Wsis[49])[1])
ax[2].set_title("n=50", fontsize = 12)
ax[2].set_xlabel('Normalised Weights', fontsize=12)

ax[3].hist(Wsis[99], np.histogram(Wsis[99])[1])
ax[3].set_title("n=100", fontsize = 12)
ax[3].set_xlabel('Normalised Weights', fontsize=12)

plt.show()

In [8]:
np.random.normal(al*Xsis[0],sig,1)

ValueError: Output size (1,) is not compatible with broadcast dimensions of inputs (1000,).

In [27]:
#SMC
T = 100 #time of filter
N = 1000 #number of particles
Xsmc = np.zeros((T,N)) #matrix of particles for 100 times
Wsmc = np.zeros((T,N)) #matrix of weight of particles for 100 times
Meansmc = np.zeros(T)
SDsmc = np.zeros(T)
Xb = np.zeros((T,N)) #matrix of resampling particles

xsmc0 = np.random.normal(0,(sig**2/(1-al**2))**0.5,N) #q(x1|y1)= u(x1) sample particle for first time
alsmc0 = norm.pdf(Ysys[0], 0, ((beta**2)*np.exp(xsmc0))**0.5)
wsmc0 = alsmc0/np.sum(alsmc0)  #w1=g(y1|x1)
Xsmc[0] = xsmc0
Wsmc[0] = wsmc0
Xb[0] = np.random.choice(Xsmc[0], N, p= Wsmc[0])

for t in range(0,T-1): 
    xsmc = np.random.normal(al*Xb[t],sig,N)  #q(xn|yn,xn-1) = f(xn|xn-1)
    alsmc = norm.pdf(Ysys[t+1], 0, ((beta**2)*np.exp(xsmc))**0.5) #calculate incremental weight \alpha_n = g(yn|xn)
    wsmc = alsmc/np.sum(alsmc) #weight of each particl at time n #wn=\alpha_n * wn-1
    Xsmc[t+1] = xsmc
    Wsmc[t+1] = wsmc
    Xb[t+1] = np.random.choice(Xsmc[t+1], N, p= Wsmc[t+1])
    
for k in range(0,T):
    weighted_stats = weighted_avg_and_std(Xsmc[k], Wsmc[k])
    Meansmc[k] = weighted_stats[0] #filtering mean time n
    SDsmc[k] = weighted_stats[1] #filtering sd time n
    
plt.figure(figsize=(15,15))
plt.title("SMC Filtering Estimates", fontsize = 20)
plt.xlabel('time',fontsize=20)
#plt.ylabel('x-coordinate', fontsize=20)
plt.plot(np.arange(start=1, stop=T+1, step=1),Xsys[0:T],marker='+',linestyle='-',color='blue',label='True Volatility')
plt.plot(np.arange(start=1, stop=T+1, step=1),Meansmc,linestyle='-',linewidth=1,color='red',label='Filter Mean')
plt.plot(np.arange(start=1, stop=T+1, step=1),Meansmc+SDsmc,linestyle='--',linewidth=0.7,color='red',label='+/- 1 S.D')
plt.plot(np.arange(start=1, stop=T+1, step=1),Meansmc-SDsmc,linestyle='--',linewidth=0.7,color='red')
plt.legend(loc='best',fontsize=15)
plt.show()

#SMC histogram of particle weights
fig, ax = plt.subplots(1,4, figsize=(15,8))
fig.suptitle('SMC')

ax[0].hist(Wsmc[1], np.histogram(Wsmc[1])[1])
ax[0].set_title("n=2", fontsize = 12)
ax[0].set_xlabel('Normalised Weights', fontsize=12)
ax[0].set_ylabel('Particle Count', fontsize=12)

ax[1].hist(Wsmc[9], np.histogram(Wsmc[9])[1])
ax[1].set_title("n=10", fontsize = 12)
ax[1].set_xlabel('Normalised Weights', fontsize=12)

ax[2].hist(Wsmc[49], np.histogram(Wsmc[49])[1])
ax[2].set_title("n=50", fontsize = 12)
ax[2].set_xlabel('Normalised Weights', fontsize=12)

ax[3].hist(Wsmc[99], np.histogram(Wsmc[99])[1])
ax[3].set_title("n=100", fontsize = 12)
ax[3].set_xlabel('Normalised Weights', fontsize=12)

plt.show()

In [28]:
#SMC with smoothing
T = 100 #time of filter
N = 1000 #number of particles
Xsmo = np.zeros((T,N)) #matrix of particles for 100 times
Wsmo = np.zeros((T,N)) #matrix of weight of particles for 100 times
Meansmo = np.zeros(T)
SDsmo = np.zeros(T)
Xb = np.zeros((T,N)) #matrix of resampling particles

xsmo0 = np.random.normal(0,(sig**2/(1-al**2))**0.5,N) #q(x1|y1)= u(x1) sample particle for first time
alsmo0 = norm.pdf(Ysys[0], 0, ((beta**2)*np.exp(xsmo0))**0.5)
wsmo0 = alsmo0/np.sum(alsmo0)  #w1=g(y1|x1)
Xsmo[0] = xsmo0
Wsmo[0] = wsmo0
Xb[0] = np.random.choice(Xsmo[0], N, p= Wsmo[0])

for t in range(0,T-1): 
    xsmo = np.random.normal(al*Xb[t],sig,N)  #q(xn|yn,xn-1) = f(xn|xn-1)
    alsmo = norm.pdf(Ysys[t+1], 0, ((beta**2)*np.exp(xsmo))**0.5) #calculate incremental weight \alpha_n = g(yn|xn)
    wsmo = alsmo/np.sum(alsmo) #weight of each particl at time n #wn=\alpha_n * wn-1
    Xsmo[t+1] = xsmo
    Wsmo[t+1] = wsmo
    Xb[t+1] = xsmo
    #resampling step
    w = np.random.choice(np.arange(0,N), N, p= Wsmo[t+1]) #to find which particle to resampling
    for k in np.arange(0,N):
        for j in np.arange(0,t+2):
            Xb[j][k] = Xb[j][w[k]]
    
for k in range(0,T):
    weighted_stats = weighted_avg_and_std(Xb[k], Wsmo[T-1])
    Meansmo[k] = weighted_stats[0] #filtering mean time n
    SDsmo[k] = weighted_stats[1] #filtering sd time n

plt.figure(figsize=(15,15))
plt.title("SMC Smoothing Estimates", fontsize = 20)
plt.xlabel('time',fontsize=20)
#plt.ylabel('x-coordinate', fontsize=20)
plt.plot(np.arange(start=1, stop=T+1, step=1),Xsys[0:T],marker='+',linestyle='-',color='blue',label='True Volatility')
plt.plot(np.arange(start=1, stop=T+1, step=1),Meansmo,linestyle='-',linewidth=1,color='red',label='Smoother Mean')
plt.plot(np.arange(start=1, stop=T+1, step=1),Meansmo+SDsmo,linestyle='--',linewidth=0.7,color='red',label='+/- 1 S.D')
plt.plot(np.arange(start=1, stop=T+1, step=1),Meansmo-SDsmo,linestyle='--',linewidth=0.7,color='red')
plt.legend(loc='best',fontsize=15)
plt.show()

#numbers of distinct particle
num = []
for c in np.arange(0,T):
    num.append(len(np.unique(Xb[c], return_counts=True)[1]))
    
plt.figure(figsize=(15,15))
plt.title("Numbers of distinct particle in time by using SMC Smoothing", fontsize = 20)
plt.xlabel('time',fontsize=20)
plt.ylabel('number', fontsize=20)
plt.plot(np.arange(start=1, stop=T+1, step=1),num,marker='o',linestyle='-',color='blue')
plt.show()

In [29]:
1/np.sum(Wsis[99]**2) #ESS

1.28237828375799